In [6]:
import tensorflow as tf

url_info = 'https://github.com/GMDays/Kinship/blob/master/NRML/NRML/nrml/nrml_train.m'
tf.__version__

'1.14.0'

In [8]:
from tensorflow.python.client import device_lib
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
tf.test.is_built_with_cuda()

Default GPU Device: /device:GPU:0


True

In [9]:
!conda env list

# conda environments:
#
base                     /home/monowar.a/anaconda3
tf-gpu-cuda9          *  /home/monowar.a/anaconda3/envs/tf-gpu-cuda9
tf_gpu_nrml              /home/monowar.a/anaconda3/envs/tf_gpu_nrml



In [10]:
# from tensorflow.python.client import device_lib
import tensorflow as tf
import numpy as np 
import pandas as pd
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from numpy.linalg import norm
import pdb

import keras.backend as K
from keras.models import load_model, Model
from keras.layers import *
from keras.optimizers import *
from keras.backend import clear_session
from keras.preprocessing.image import ImageDataGenerator as IDG
from keras.utils import Sequence
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.losses import *

# import tensorflow.keras.backend as K
# from tensorflow.keras.models import load_model, Model
# from tensorflow.keras.layers import *
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.backend import clear_session
# from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
# from tensorflow.keras.utils import Sequence
# from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.callbacks import TensorBoard

from PIL import Image
import matplotlib.pyplot as plt
import os

from PIL import Image
import os


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
config = tf.ConfigProto()
config = tf.ConfigProto( device_count = {'GPU': 2 , 'CPU': 1} )
config.gpu_options.allow_growth = True
config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
session = tf.Session(config=config)
K.set_session(session)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1413097744693103260
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9555004386842362810
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10818532148
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 2
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 4062803833199052138
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10818532148
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 2
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 12124616745306904436
physical_d

Using TensorFlow backend.


In [11]:
from keras import backend as K
tb_path = "./logs/"
# Get the sessions graph
graph = K.get_session().graph

# Display with the tensorflow file writer
# writer = tf.summary.FileWriter(logdir=tb_path, graph=graph)
# writer.flush()

W0725 16:10:24.855671 140283755095808 deprecation_wrapper.py:119] From /home/monowar.a/anaconda3/envs/tf-gpu-cuda9/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0725 16:10:24.857640 140283755095808 deprecation_wrapper.py:119] From /home/monowar.a/anaconda3/envs/tf-gpu-cuda9/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [12]:
tcb = TensorBoard(log_dir='./Graph', histogram_freq=1,  
          write_graph=True, write_images=True)

In [13]:
tf.__version__

'1.14.0'

In [14]:
!nvidia-smi

Thu Jul 25 16:10:27 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   48C    P2    61W / 250W |    312MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   48C    P2    61W / 250W |    312MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [15]:
def dist_mat(P1, P2 = None):
    if P2 is not None:
        X1 = tf.tile(tf.expand_dims(tf.reduce_sum(P1**2, axis= -1), axis= -1), [1, tf.shape(P2)[0]])
        X2 = tf.tile(tf.expand_dims(tf.reduce_sum(P2**2, axis= -1), axis= -1), [1, tf.shape(P1)[0]])
        R = P1 @ tf.transpose(P2)
        D = X1 + tf.transpose(X2) - 2*R
        D = tf.sqrt(D)
        return D
    else:
        X1 = tf.tile(tf.expand_dims(tf.reduce_sum(P1**2, axis= -1), axis= -1), [1, tf.shape(P1)[0]])
        R = P1 @ tf.transpose(P1)
        D = X1 + tf.transpose(X1) - 2*R
        D = tf.real(tf.sqrt(tf.cast(D, dtype=tf.complex64)))
        return D

In [16]:
class NRML:
    def __init__(self, N, K, d):
        self.N = N
        self.K = K
        self.convergence_data = []
        self.W = 0
        self.Wt = tf.Variable(np.zeros(shape = (d,d), dtype = np.float32))
        self.X = tf.placeholder(tf.float32, shape=[N, d])
        self.Y = tf.placeholder(tf.float32, shape=[N, d])

    def fit(self, X, Y, T = 10, plot_convergence = False, epsilon = 0.0015):
        deltas = []
        W = 0
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for r in tqdm(range(T)):
                sess.run(self.step, {self.X: X, self.Y: Y})
                Wt = sess.run(self.Wt)
                deltas.append(np.mean(np.abs(Wt - W)))
                X = X @ Wt
                Y = Y @ Wt
                W = Wt
                if r>3 and deltas[-1] < epsilon:
                    break
        self.convergence_data = deltas
        self.W = W
        return (W, deltas)

    def build_model(self):
        X = self.X
        Y = self.Y      
        with tf.variable_scope('nrml'):
            with tf.variable_scope('dist_H1'):
                H1 = self.__diff(X, Y)/(self.K*self.N)
            with tf.variable_scope('dist_H2'):
                H2 = self.__diff(Y, X)/(self.K*self.N)
            with tf.variable_scope('dist_H3'):
                D = X - Y
                H3 = (tf.transpose(D) @ D)/self.N
                
            H = (H1 + H2 - H3)

            e, Wt = tf.linalg.eigh(H)
            Wt = tf.reverse(Wt, axis=[-1])
            self.step = self.Wt.assign(Wt)
        return (X, Y, self.Wt)
    
    def __diff(self, x, y):
        D = dist_mat(y)
        I = tf.argsort(D)
        y_knn_idx = I[:, 1:self.K+1]
        yt = tf.stack([tf.gather(y, i) for i in tf.unstack(y_knn_idx)], 1)
        x = tf.expand_dims(x, 0)
        d = x - yt
        return tf.reduce_sum(tf.transpose(d, perm = [0, 2, 1]) @ d, axis = 0)
    
    def plot_convergence_curve(self):
            data = pd.DataFrame({'Iterations' : range(len(self.convergence_data)), 
                                 '$\Delta$' : self.convergence_data})
            sns.lineplot(x = 'Iterations', 
                         y = '$\Delta$', 
                         data = data)

In [17]:
from keras.preprocessing.image import apply_affine_transform
def get_files_in_dir(directory):
    dirs = []
    for path, subdirs, files in os.walk(directory):
        for name in files:
            dirs.append(os.path.join(path, name))
    return dirs

def get_image_samples(directory_list, num_samples = 32, img_size = (160,160)):
    C = np.random.choice(class_dirs, size=num_samples)
    I = [np.random.choice(get_files_in_dir(j)) for j in C]
    params = {
        'theta': np.random.random()*10 - 5,
        'tx': np.random.random()*20 - 10,
        'ty': np.random.random()*20 - 10
    }
    return np.array([apply_affine_transform(np.array(Image.open(i).resize(img_size)), **params) for i in I])

In [18]:
import os
data_dir = '../NorthEastern_SMILE_dataset/Data/'
tr_dir = '../NorthEastern_SMILE_dataset/Data/Train/'
class_dirs = [tr_dir + j for j in os.listdir(tr_dir)]
img_dirs = get_files_in_dir(tr_dir)
tr_df = pd.read_csv('filtered_train_relationships.csv')

class TripletSequence(Sequence):

    def __init__(self, tr_dir, tr_df, W, model, batch_size=16, Nt=8, K = 4):
        self.tr_dir, self.tr_df = tr_dir, tr_df
        self.batch_size, self.Nt, self.K = batch_size, Nt, K
        self.W = W
        self.model = model

    def __len__(self):
        return 100

    def __getitem__(self, idx):
        batch_df = self.tr_df.sample(self.batch_size, replace=False)
        anchors, positives, negatives = [], [], []
        for i in tnrange(self.batch_size):
            anchor_dir = batch_df.iloc[i]['p1']
            pos_dir = batch_df.iloc[i]['p2']
            anchor_img = get_image_samples(self.tr_dir + anchor_dir, num_samples=1)
            pos_img = get_image_samples(self.tr_dir + pos_dir, num_samples=1)
            anchors.append(anchor_img[0])
            positives.append(pos_img[0])
        inp = np.array(anchors + positives)
        out = feature_model.predict(inp, batch_size=100, verbose=1)
        an, pos = out[:self.batch_size,:], out[self.batch_size:,:]
        an = an@self.W
        pos = pos@self.W
#         for i in tnrange(self.batch_size):
#             pool = np.concatenate([an[:i, :], an[i+1:, :], pos[:i, :], pos[i+1:, :]])
#             pool_img = anchors[:i] + anchors[i+1:] + positives[:i] + positives[i+1:]
#             ind = np.random.choice(len(pool_img), size=self.Nt, replace=False)
#             neg = np.array([pool[i] for i in ind])
#             neg_img = pool_img[np.argmin(norm(neg - an[i,:], axis=-1, ord=2))]
#             negatives.append(neg_img)
        anchors = np.array(anchors)
        positives = np.array(positives)
#         negatives = np.array(negatives)
        return [[anchors, positives], [an, pos]]

In [24]:
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from functools import partial
ALPHA = 0.2
# K.clear_session()

def arr_metric(i, inputs, outputs):
    return outputs[i]

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)

    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum((a - p)**2, axis=-1)
        n_dist = K.sum((a - n)**2, axis=-1)
        return K.mean(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)

    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape, list)
        return (None,)


def build_facenet_model(embedding_network, classifier):
    input_shape = embedding_network.input_shape[1:]
    embedding_network.layers.pop(0)
    embedding_network.layers[0]._inbound_nodes = []
    positive_sample = Input(shape=input_shape)
    negative_sample = Input(shape=input_shape)
    anchor_sample = Input(shape=input_shape)
    
    embedding_p = embedding_network(positive_sample)
    embedding_n = embedding_network(negative_sample)
    embedding_a = embedding_network(anchor_sample)
    triplet_loss = TripletLossLayer(alpha=0.2, 
                                          name='triplet_loss_layer')([embedding_a, embedding_p, embedding_n])
    model = Model(inputs=[anchor_sample, positive_sample, negative_sample],
                  outputs=triplet_loss)
    
    model.compile(optimizer="adam", loss=None)
    return model

def build_classifier_model():
    x1 = Input(shape=(128,))
    x2 = Input(shape=(128,))
    x = Lambda(lambda _x: K.concatenate([K.expand_dims(i, -1) for i in _x], axis = -1))([x1, x2])
    x = Lambda(lambda _x: K.expand_dims(x, -1))(x)
    x = Conv2D(64,kernel_size=2, activation = 'relu', padding = 'same')(x)
    x = MaxPool2D((8,1))(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(32, name ='classifier_dense_1', 
              activation='relu', 
              use_bias=False, 
              kernel_initializer='VarianceScaling', kernel_regularizer='l2')(x)
    x = Dropout(rate=0.2)(x)
    x = BatchNormalization()(x)
    x = Dense(8, name ='classifier_dense_2', 
              activation='relu', 
              use_bias=False, 
              kernel_initializer='VarianceScaling', kernel_regularizer='l2')(x)
    x = BatchNormalization()(x)
    c = Dense(1, name ='classifier_dense_3', 
              activation='sigmoid', 
              use_bias=False, 
              kernel_initializer='VarianceScaling', kernel_regularizer='l2')(x)
    m = Model(inputs = [x1, x2], outputs = c)
    m.compile(optimizer="adam",loss=binary_crossentropy, metrics = ['acc'])
    return m

def build_feature_model():
    input_tensor = Input(shape=(160, 160, 3))
#     base_model = InceptionResNetV2(input_tensor=input_tensor, weights='imagenet', include_top=False)
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Dropout(rate=0.2)(x)
    base_model = load_model('facenet_keras.h5')
    x = base_model.layers[-1].output
#     x = Dense(386, name ='Bottleneck', 
#               activation='linear', 
#               use_bias=True, 
#               kernel_initializer='VarianceScaling')(x)
#     x = BatchNormalization(name='Bottleneck_BatchNorm')(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer= Adam(0.00075) ,loss='binary_crossentropy')
    return model

classifier = build_classifier_model()
feature_model = build_feature_model()
model = build_facenet_model(feature_model, classifier)
model.summary()


/home/monowar.a/anaconda3/envs/tf-gpu-cuda9/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 128)          22808144    input_10[0][0]                   
                                                                 input_11[0][0]                   
          

In [25]:
from keras_tqdm import TQDMNotebookCallback
train = TripletSequence(tr_dir=tr_dir, tr_df=tr_df[768:], W = np.eye(128), model=feature_model, batch_size=2048)
val = TripletSequence(tr_dir=tr_dir, tr_df=tr_df[:768], W = np.eye(128), model=feature_model, batch_size=512)
nrml = NRML(N=2048, K=17, d=128)
nrml.build_model()

num_runs = 15

for i in range(num_runs):
    print('Training Loop {0}'.format(i))
    print('Fetching Data: ')
    [[anchors, positives], [x, y]] = train.__getitem__(1)
    [[vanchors, vpositives], [vx, vy]] = val.__getitem__(1)
    
    ones = np.ones((2*len(x),))
    zeros = np.zeros((2*len(x),))
    opt = np.concatenate([ones, zeros])
    c_trx = np.concatenate([x, y, x, y[::-1,:]])
    c_try = np.concatenate([y, x, y[::-1,:], x])
    
    vones = np.ones((2*len(vx)))
    vzeros = np.zeros((2*len(vx)))
    vopt = np.concatenate([vones, vzeros])
    vc_trx = np.concatenate([vx, vy, vx,         vy[::-1,:]])
    vc_try = np.concatenate([vy, vx, vy[::-1,:], vx])
    print('Fitting Classifier: ')
    classifier.fit(x = [c_trx, c_try], y = [opt], shuffle=True,
                   validation_data = ([vc_trx, vc_try], [vopt]),
                   batch_size=128,
                   verbose = 1, epochs=20)
    del vc_trx, vc_try, vopt, vones, vzeros, c_trx, c_try, opt, ones, zeros
    negatives = []
    pool_img =np.concatenate([anchors, positives])
    my_pool = np.concatenate([y, x])
    for i in tnrange(len(x)):
        ind = list(range(len(pool_img)))
        ind.pop(i)
        ind.pop(len(x)+i-1)
        ind = np.random.choice(ind, size = (256,))
        mx = [np.expand_dims(x[i,:],0)]
        mx = np.concatenate(len(ind)*mx)
        pool = pool_img[ind,:,:,:]
        my = my_pool[ind,:]
        myp = classifier.predict([mx, my])
        neg_img = pool_img[np.argmax(myp)]
        negatives.append(neg_img)
    negatives = np.array(negatives)
    print('Fitting Feature Model: ')
    model.fit(x = [anchors, positives, negatives], y = [],
              callbacks=[], 
              batch_size=32, 
              verbose = 1, epochs=7)
    print('Fine tuning Distance matrix: ')
    nrml.fit(x, y, 100, .00075)
    if i != num_runs - 1:
        train.W = nrml.W
    print('---------------------------------------------------------')
    print('\n\n')

Training Loop 0
Fetching Data: 


4096/4096 [==============================] - 24s 6ms/step


1024/1024 [==============================] - 1s 842us/step
Fitting Classifier: 
Train on 8192 samples, validate on 2048 samples
Epoch 1/20
8192/8192 [==============================] - 17s 2ms/step - loss: 1.0892 - acc: 0.4927 - val_loss: 0.9784 - val_acc: 0.5029
Epoch 2/20
8192/8192 [==============================] - 1s 111us/step - loss: 0.9159 - acc: 0.5001 - val_loss: 0.8641 - val_acc: 0.4932
Epoch 3/20
8192/8192 [==============================] - 1s 121us/step - loss: 0.8343 - acc: 0.4995 - val_loss: 0.8048 - val_acc: 0.5044
Epoch 4/20
8192/8192 [==============================] - 1s 113us/step - loss: 0.7894 - acc: 0.5017 - val_loss: 0.7711 - val_acc: 0.5039
Epoch 5/20
8192/8192 [==============================] - 1s 119us/step - loss: 0.7629 - acc: 0.4983 - val_loss: 0.7513 - val_acc: 0.5010
Epoch 6/20
8192/8192 [==============================] - 1s 115us/step - loss: 0.7456 - acc: 0.4966 - val_loss: 0.7371 - val_acc: 0.4902
Epoch 7/20
8192/8192 [==============================] - 1

Fitting Feature Model: 
Epoch 1/7
2048/2048 [==============================] - 145s 71ms/step - loss: 42.3957
Epoch 2/7
2048/2048 [==============================] - 22s 11ms/step - loss: 37.3602
Epoch 3/7
2048/2048 [==============================] - 22s 11ms/step - loss: 35.5965
Epoch 4/7
2048/2048 [==============================] - 22s 11ms/step - loss: 31.8477
Epoch 5/7
2048/2048 [==============================] - 22s 11ms/step - loss: 23.6537
Epoch 6/7
2048/2048 [==============================] - 22s 11ms/step - loss: 16.8286
Epoch 7/7
2048/2048 [==============================] - 22s 11ms/step - loss: 13.5817
Fine tuning Distance matrix: 


---------------------------------------------------------



Training Loop 1
Fetching Data: 


...

---------------------------------------------------------



Training Loop 14
Fetching Data: 


4096/4096 [==============================] - 3s 809us/step


1024/1024 [==============================] - 1s 809us/step
Fitting Classifier: 
Train on 8192 samples, validate on 2048 samples
Epoch 1/20
8192/8192 [==============================] - 1s 103us/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/20
8192/8192 [==============================] - 1s 121us/step - loss: 0.6931 - acc: 0.4934 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/20
8192/8192 [==============================] - 1s 124us/step - loss: 0.6931 - acc: 0.4988 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 4/20
8192/8192 [==============================] - 1s 125us/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 5/20
8192/8192 [==============================] - 1s 127us/step - loss: 0.6931 - acc: 0.4941 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 6/20
8192/8192 [==============================] - 1s 127us/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 7/20
8192/8192 [==============================] - 

Fitting Feature Model: 
Epoch 1/7
2048/2048 [==============================] - 22s 11ms/step - loss: 23.2956
Epoch 2/7
2048/2048 [==============================] - 22s 11ms/step - loss: 17.4702
Epoch 3/7
2048/2048 [==============================] - 22s 11ms/step - loss: 14.8208
Epoch 4/7
2048/2048 [==============================] - 22s 11ms/step - loss: 11.6182
Epoch 5/7
2048/2048 [==============================] - 22s 11ms/step - loss: 10.1120
Epoch 6/7
2048/2048 [==============================] - 22s 11ms/step - loss: 8.4420
Epoch 7/7
2048/2048 [==============================] - 22s 11ms/step - loss: 5.8502
Fine tuning Distance matrix: 


---------------------------------------------------------





In [ ]:
model.save_weights('model.h5')

In [ ]:
[[anchors, positives, negatives], [x, y]] = train.__getitem__(1)

In [ ]:
y_bar = feature_model.predict(negatives, verbose = 1)

In [ ]:
K.eval(K.sum((x-y)**2, -1))

In [ ]:
K.eval(K.sum((x-y_bar)**2, -1))

In [ ]:
res = K.eval(K.sum((x-y)**2, -1)) < K.eval(K.sum((x-y_bar)**2, -1))

In [ ]:
len(res)

In [ ]:
np.sum(res)

In [ ]:
def kld(y_true, y_pred):
    y_true = np.clip(y_true, np.finfo(float).eps, 1)
    y_pred = np.clip(y_pred, np.finfo(float).eps, 1)
    return np.sum(y_true * np.log(y_true / y_pred), axis=-1)

res = kld(x,y) < kld(x,y_bar)